In [15]:
import msgpack

with open('SQuAD/data.msgpack', 'rb') as f:
    data = msgpack.load(f, encoding='utf8')
# train = sorted(data['train'], key=lambda x: len(x[1]))
# dev = data['dev'].sort(key=lambda x: len(x[1]))


In [16]:
# train: id, context_id, context_features, tag_id, ent_id,
#        question_id, context, context_token_span, answer_start, answer_end
# dev:   id, context_id, context_features, tag_id, ent_id,
#        question_id, context, context_token_span, answer
import torch
import re
import traceback
def train_data_trans(row):
    idx, context_id, context_features, tag_id, ent_id,\
    question_id, context, context_token_span, answer_start, answer_end = row
    return len(context_id), len(question_id), idx, torch.tensor(context_id), torch.tensor(question_id), \
           [torch.tensor(context_id[answer_start:answer_end + 1])], [torch.tensor([answer_start, answer_end])]
def train_data_dev(row):
    idx, context_id, context_features, tag_id, ent_id,\
    question_id, context, context_token_span, answers = row
    # try:
    #     for answer in answers:
    #         answer_match = re.search(answer, context)
    #         if answer_match is not None:
    #             answer_start = list(zip(*context_token_span))[0].index(answer_match.start())
    #             answer_end = list(zip(*context_token_span))[1].index(answer_match.end())
    #             answer_list.append(torch.tensor(context_id[answer_start:answer_end + 1]))
    #             span_list.append(torch.tensor([answer_start, answer_end]))
    return len(context_id), len(question_id), idx, torch.tensor(context_id), torch.tensor(question_id), \
           answers
    # except:
    #     print(idx)
    #     print(answer_match)
    #     print(context_token_span)
    #     traceback.print_exc()

In [17]:
# from pre import flatten_json
# train_json = flatten_json('.data/train-v1.1.json', 'train')
# dev_json = flatten_json('.data/dev-v1.1.json', 'dev')

In [18]:
# dev_0 = data['dev'][1]
# # t_json_0 = train_json[0]
# trans_0 = train_data_dev(dev_0)

In [19]:
trans_train = sorted([train_data_trans(row) for row in data['train']])
trans_dev = sorted([train_data_dev(row) for row in data['dev']])

In [20]:
import pickle
with open('trans_train.pickle', 'wb') as f:
    pickle.dump(trans_train, f, pickle.HIGHEST_PROTOCOL)
with open('trans_dev.pickle', 'wb') as f:
    pickle.dump(trans_dev, f, pickle.HIGHEST_PROTOCOL)

In [21]:
dev_wrong = [item for item in data['dev']
             if item[0] == '56e12005cd28a01900c67619'][0]
train_data_dev(dev_wrong)

(47,
 7,
 '56e12005cd28a01900c67619',
 tensor([  737,   148,    34,  7480,    31,   352,     6,  5458,  2032, 16465,
            19, 26568,    90,     3,    50,    15,    83,    56,  4444,     9,
          5378,   181,     4,   113,     1,   145,    13,  7563,     8, 13194,
          1893,    44,    24,   154,  4076,     3,   737,  2138,     9,  7529,
             8,  1236,   224,     6,   491,   113,     1]),
 tensor([1089,   49,   56, 1231,    2, 5378,   18]),
 ['her weight', 'her weight', 'her weight.'])

In [22]:
dev_wrong[6]

'Tesla could be harsh at times and openly expressed disgust for overweight people, such as when he fired a secretary because of her weight.:110 He was quick to criticize clothing; on several occasions, Tesla directed a subordinate to go home and change her dress.:33'